# **Problem Statement**

An education company named X Education sells online courses to industry professionals. On any given day, many professionals who are interested in the courses land on their website and browse for courses. 

 

The company markets its courses on several websites and search engines like Google. Once these people land on the website, they might browse the courses or fill up a form for the course or watch some videos. When these people fill up a form providing their email address or phone number, they are classified to be a lead. Moreover, the company also gets leads through past referrals. Once these leads are acquired, employees from the sales team start making calls, writing emails, etc. Through this process, some of the leads get converted while most do not. The typical lead conversion rate at X education is around 30%. 

 

Now, although X Education gets a lot of leads, its lead conversion rate is very poor. For example, if, say, they acquire 100 leads in a day, only about 30 of them are converted. To make this process more efficient, the company wishes to identify the most potential leads, also known as ‘Hot Leads’. If they successfully identify this set of leads, the lead conversion rate should go up as the sales team will now be focusing more on communicating with the potential leads rather than making calls to everyone.

X Education has appointed you to help them select the most promising leads, i.e. the leads that are most likely to convert into paying customers. The company requires you to build a model wherein you need to assign a lead score to each of the leads such that the customers with higher lead score have a higher conversion chance and the customers with lower lead score have a lower conversion chance. The CEO, in particular, has given a ballpark of the target lead conversion rate to be around 80%.

**Goals of the Case Study**

 - Build a logistic regression model to assign a lead score between 0 and 100 to each of the leads which can be used by the company to target potential leads. A higher score would mean that the lead is hot, i.e. is most likely to convert whereas a lower score would mean that the lead is cold and will mostly not get converted.
 

 - There are some more problems presented by the company which your model should be able to adjust to if the company's requirement changes in the future so you will need to handle these as well. These problems are provided in a separate doc file. Please fill it based on the logistic regression model you got in the first step. Also, make sure you include this in your final PPT where you'll make recommendations.

# **Segment - I : Data Understanding**

## Importing the libraries and reading the dataset

In [1]:
# Importing the useful libraries and functions

# Suppress Warnings

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# Import Numpy and Pandas Library

import pandas as pd
import numpy as np

# Import Matplotlib and Seaborn Libraries for Data visualization

import matplotlib.pyplot as plt
import seaborn as sns

# Using Pandas Inbuilt function to display all columns and rows present in Dataset

pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.max_rows', None)

# Using sklearn library and importing train_test_split function to create Training & Test Datasets

from sklearn.model_selection import train_test_split

# Using sklearn library and importing MinMaxScaler function for feature scaling

from sklearn.preprocessing import MinMaxScaler

# Using sklearn library and importing RFE and LogisticRegression

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Importing statsmodel library

import statsmodels.api as sm

In [2]:
# Reading the dataset

leads = pd.read_csv("Leads.csv")

In [3]:
# Printing the first five rows of the dataframe 'leads'

leads.head()

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,Specialization,How did you hear about X Education,What is your current occupation,What matters most to you in choosing a course,Search,Magazine,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Receive More Updates About Our Courses,Tags,Lead Quality,Update me on Supply Chain Content,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,Page Visited on Website,NaN,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Interested in other courses,Low in Relevance,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,Email Opened,India,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,NaN,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,Email Opened,India,Business Administration,Select,Student,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,Unreachable,India,Media and Advertising,Word Of Mouth,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,Not Sure,No,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,Converted to Lead,India,Select,Other,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified


## Data inspection

### Basic structure checking

In [4]:
# Checking the shape of dataframe

leads.shape

(9240, 37)

**There are 9240 rows and 37 columns initially.**

In [5]:
# Printing information about the attributes of 'leads'

leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 37 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Prospect ID                                    9240 non-null   object 
 1   Lead Number                                    9240 non-null   int64  
 2   Lead Origin                                    9240 non-null   object 
 3   Lead Source                                    9204 non-null   object 
 4   Do Not Email                                   9240 non-null   object 
 5   Do Not Call                                    9240 non-null   object 
 6   Converted                                      9240 non-null   int64  
 7   TotalVisits                                    9103 non-null   float64
 8   Total Time Spent on Website                    9240 non-null   int64  
 9   Page Views Per Visit                           9103 

In [6]:
# Checking the size of dataframe

leads.size

341880

In [7]:
# Checking datatypes of all columns

leads.dtypes


Prospect ID                                       object
Lead Number                                        int64
Lead Origin                                       object
Lead Source                                       object
Do Not Email                                      object
Do Not Call                                       object
Converted                                          int64
TotalVisits                                      float64
Total Time Spent on Website                        int64
Page Views Per Visit                             float64
Last Activity                                     object
Country                                           object
Specialization                                    object
How did you hear about X Education                object
What is your current occupation                   object
What matters most to you in choosing a course     object
Search                                            object
Magazine                       

In [8]:
# Finding the basic statistical values for numerical data

leads.describe()

,Lead Number,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Asymmetrique Activity Score,Asymmetrique Profile Score
count,9240.000000,9240.000000,9103.000000,9240.000000,9103.000000,5022.000000,5022.000000
mean,617188.435606,0.385390,3.445238,487.698268,2.362820,14.306252,16.344883
std,23405.995698,0.486714,4.854853,548.021466,2.161418,1.386694,1.811395
min,579533.000000,0.000000,0.000000,0.000000,0.000000,7.000000,11.000000
25%,596484.500000,0.000000,1.000000,12.000000,1.000000,14.000000,15.000000
50%,615479.000000,0.000000,3.000000,248.000000,2.000000,14.000000,16.000000
75%,637387.250000,1.000000,5.000000,936.000000,3.000000,15.000000,18.000000
max,660737.000000,1.000000,251.000000,2272.000000,55.000000,18.000000,20.000000


### Checking missing values and duplicated rows

In [9]:
# Calculating the missing values in each column of the dataframe

leads.isnull().sum()

Prospect ID                                         0
Lead Number                                         0
Lead Origin                                         0
Lead Source                                        36
Do Not Email                                        0
Do Not Call                                         0
Converted                                           0
TotalVisits                                       137
Total Time Spent on Website                         0
Page Views Per Visit                              137
Last Activity                                     103
Country                                          2461
Specialization                                   1438
How did you hear about X Education               2207
What is your current occupation                  2690
What matters most to you in choosing a course    2709
Search                                              0
Magazine                                            0
Newspaper Article           

In [10]:
# Calculating the percentage of missing values in each column

round(100*(leads.isnull().sum())/len(leads), 2)

Prospect ID                                       0.00
Lead Number                                       0.00
Lead Origin                                       0.00
Lead Source                                       0.39
Do Not Email                                      0.00
Do Not Call                                       0.00
Converted                                         0.00
TotalVisits                                       1.48
Total Time Spent on Website                       0.00
Page Views Per Visit                              1.48
Last Activity                                     1.11
Country                                          26.63
Specialization                                   15.56
How did you hear about X Education               23.89
What is your current occupation                  29.11
What matters most to you in choosing a course    29.32
Search                                            0.00
Magazine                                          0.00
Newspaper 

In [11]:
# Checking for duplicated rows

leads.duplicated().sum()

0

In [12]:
leads.shape,leads.drop_duplicates().shape

((9240, 37), (9240, 37))

***Observation:***

**There are no duplicated rows in this dataframe. The same is verified by checking the shape of the dataframe before and after dropping duplicates, if any.**

## **Data cleaning**


In [13]:
leads.columns

Index(['Prospect ID', 'Lead Number', 'Lead Origin', 'Lead Source',
       'Do Not Email', 'Do Not Call', 'Converted', 'TotalVisits',
       'Total Time Spent on Website', 'Page Views Per Visit', 'Last Activity',
       'Country', 'Specialization', 'How did you hear about X Education',
       'What is your current occupation',
       'What matters most to you in choosing a course', 'Search', 'Magazine',
       'Newspaper Article', 'X Education Forums', 'Newspaper',
       'Digital Advertisement', 'Through Recommendations',
       'Receive More Updates About Our Courses', 'Tags', 'Lead Quality',
       'Update me on Supply Chain Content', 'Get updates on DM Content',
       'Lead Profile', 'City', 'Asymmetrique Activity Index',
       'Asymmetrique Profile Index', 'Asymmetrique Activity Score',
       'Asymmetrique Profile Score',
       'I agree to pay the amount through cheque',
       'A free copy of Mastering The Interview', 'Last Notable Activity'],
      dtype='object')

### Checking value counts for each categorical column

In [14]:
var_list = ['Lead Source', 'Do Not Email', 'Do Not Call', 'Converted', 'Last Activity', 'Country', 'Specialization',
            'How did you hear about X Education', 'What is your current occupation', 
            'What matters most to you in choosing a course', 'Search', 'Magazine', 'Newspaper Article', 
            'X Education Forums', 'Newspaper', 'Digital Advertisement', 'Through Recommendations',
            'Receive More Updates About Our Courses', 'Tags', 'Lead Quality', 'Update me on Supply Chain Content', 
            'Get updates on DM Content', 'Lead Profile', 'City', 'Asymmetrique Activity Index', 'Asymmetrique Profile Index', 
            'I agree to pay the amount through cheque', 'A free copy of Mastering The Interview', 'Last Notable Activity']


for i in var_list:
    ans=leads[i].value_counts(dropna=False)
    print(f"{i} Count:\n\n{ans}",end='\n\n\n')
    

Lead Source Count:

Google               2868
Direct Traffic       2543
Olark Chat           1755
Organic Search       1154
Reference             534
Welingak Website      142
Referral Sites        125
Facebook               55
NaN                    36
bing                    6
google                  5
Click2call              4
Press_Release           2
Social Media            2
Live Chat               2
youtubechannel          1
testone                 1
Pay per Click Ads       1
welearnblog_Home        1
WeLearn                 1
blog                    1
NC_EDM                  1
Name: Lead Source, dtype: int64


Do Not Email Count:

No     8506
Yes     734
Name: Do Not Email, dtype: int64


Do Not Call Count:

No     9238
Yes       2
Name: Do Not Call, dtype: int64


Converted Count:

0    5679
1    3561
Name: Converted, dtype: int64


Last Activity Count:

Email Opened                    3437
SMS Sent                        2745
Olark Chat Conversation          973
Page Visited 

**Observation:**

**There are categories named 'Select' in columns like 'Specialization', 'How did you hear about X Education', 'Lead Profile', etc. This is because the site visitors have not selected any response for this and it is as good as NaN or a null value. Hence, replacing 'Select' with NaN.**

### Replacing 'Select' with NaN

In [15]:
# Replacing 'Select' with NaN since no option is selected for these values

leads = leads.replace('Select',np.nan)

### Dropping columns having more than 40% null values

In [16]:
# Finding missing value percencentage in each column

null_percent = round(100*(leads.isnull().sum())/len(leads), 2)
print(null_percent)

# List of columns having more than 40% null values.

null_percent[null_percent > 40].index

Prospect ID                                       0.00
Lead Number                                       0.00
Lead Origin                                       0.00
Lead Source                                       0.39
Do Not Email                                      0.00
Do Not Call                                       0.00
Converted                                         0.00
TotalVisits                                       1.48
Total Time Spent on Website                       0.00
Page Views Per Visit                              1.48
Last Activity                                     1.11
Country                                          26.63
Specialization                                   36.58
How did you hear about X Education               78.46
What is your current occupation                  29.11
What matters most to you in choosing a course    29.32
Search                                            0.00
Magazine                                          0.00
Newspaper 

Index(['How did you hear about X Education', 'Lead Quality', 'Lead Profile',
       'Asymmetrique Activity Index', 'Asymmetrique Profile Index',
       'Asymmetrique Activity Score', 'Asymmetrique Profile Score'],
      dtype='object')

**Approach :**

1. Dropping all the columns that have extremely high missing values. Considering that value as 40% in this case study.

In [17]:
# Dropping columns having more than 40% of null values

leads = leads.drop(null_percent[null_percent > 40].index , axis= 1)

# Checking the shape again after dropping columns
leads.shape

(9240, 30)

### Checking the number of unique values in each column

In [18]:
# Finding the number of unique values in each column

leads.nunique()

Prospect ID                                      9240
Lead Number                                      9240
Lead Origin                                         5
Lead Source                                        21
Do Not Email                                        2
Do Not Call                                         2
Converted                                           2
TotalVisits                                        41
Total Time Spent on Website                      1731
Page Views Per Visit                              114
Last Activity                                      17
Country                                            38
Specialization                                     18
What is your current occupation                     6
What matters most to you in choosing a course       3
Search                                              2
Magazine                                            1
Newspaper Article                                   2
X Education Forums          

**Observation:**

1. There are columns which have just one unique value and these will not play any significant role in further analysis. Hence, dropping those columns.

In [19]:
# Dropping unique valued columns

leads= leads.drop(['Magazine','Receive More Updates About Our Courses','Update me on Supply Chain Content',
                   'Get updates on DM Content','I agree to pay the amount through cheque'],axis=1)
leads.shape

(9240, 25)

In [20]:
leads.head()

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,Specialization,What is your current occupation,What matters most to you in choosing a course,Search,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Tags,City,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,Page Visited on Website,NaN,NaN,Unemployed,Better Career Prospects,No,No,No,No,No,No,Interested in other courses,NaN,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,Email Opened,India,NaN,Unemployed,Better Career Prospects,No,No,No,No,No,No,Ringing,NaN,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,Email Opened,India,Business Administration,Student,Better Career Prospects,No,No,No,No,No,No,Will revert after reading the email,Mumbai,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,Unreachable,India,Media and Advertising,Unemployed,Better Career Prospects,No,No,No,No,No,No,Ringing,Mumbai,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,Converted to Lead,India,NaN,Unemployed,Better Career Prospects,No,No,No,No,No,No,Will revert after reading the email,Mumbai,No,Modified


### Dropping unnecassary columns

In [21]:
# Checking rows which have all NaN values.

leads[leads.isnull().all(axis=1)].shape

(0, 25)

**There are no rows which have all null values present.**

**'Prospect ID' and 'Lead Number' is the unique ID and a lead number assigned to each lead, with which the customer is identified. These are all unique values, hence dropping these columns as they will not provide any insight to the analysis.**

In [22]:
# Dropping 'Lead Number' and 'Prospect ID'.

leads.drop(['Prospect ID', 'Lead Number'], 1, inplace = True)
leads.shape

(9240, 23)

In [23]:
# Checking null value percentage in each column

round(100*(leads.isnull().sum())/len(leads),2)

Lead Origin                                       0.00
Lead Source                                       0.39
Do Not Email                                      0.00
Do Not Call                                       0.00
Converted                                         0.00
TotalVisits                                       1.48
Total Time Spent on Website                       0.00
Page Views Per Visit                              1.48
Last Activity                                     1.11
Country                                          26.63
Specialization                                   36.58
What is your current occupation                  29.11
What matters most to you in choosing a course    29.32
Search                                            0.00
Newspaper Article                                 0.00
X Education Forums                                0.00
Newspaper                                         0.00
Digital Advertisement                             0.00
Through Re

**Country**

In [24]:
# Checking the unique categories in 'Country'

leads['Country'].value_counts(normalize=True,dropna=False)

India                   0.702597
NaN                     0.266342
United States           0.007468
United Arab Emirates    0.005736
Singapore               0.002597
Saudi Arabia            0.002273
United Kingdom          0.001623
Australia               0.001407
Qatar                   0.001082
Bahrain                 0.000758
Hong Kong               0.000758
Oman                    0.000649
France                  0.000649
unknown                 0.000541
Kuwait                  0.000433
South Africa            0.000433
Canada                  0.000433
Nigeria                 0.000433
Germany                 0.000433
Sweden                  0.000325
Philippines             0.000216
Uganda                  0.000216
Italy                   0.000216
Bangladesh              0.000216
Netherlands             0.000216
Asia/Pacific Region     0.000216
China                   0.000216
Belgium                 0.000216
Ghana                   0.000216
Kenya                   0.000108
Sri Lanka 

**It can be seen, more than 70% of the entries are from 'India' and more than 26% are missing values, hence dropping this column as it will not provide anything significant in the analysis.**

In [25]:
# Dropping column 'Country'

leads.drop(['Country'], 1, inplace = True)
leads.shape

(9240, 22)

**Specialization**

In [26]:
# Checking the unique categories in 'Specialization'

leads['Specialization'].value_counts(normalize=True,dropna=False)

NaN                                  0.365801
Finance Management                   0.105628
Human Resource Management            0.091775
Marketing Management                 0.090693
Operations Management                0.054437
Business Administration              0.043615
IT Projects Management               0.039610
Supply Chain Management              0.037771
Banking, Investment And Insurance    0.036580
Travel and Tourism                   0.021970
Media and Advertising                0.021970
International Business               0.019264
Healthcare Management                0.017208
Hospitality Management               0.012338
E-COMMERCE                           0.012121
Retail Management                    0.010823
Rural and Agribusiness               0.007900
E-Business                           0.006169
Services Excellence                  0.004329
Name: Specialization, dtype: float64

In [27]:
leads['Specialization'] = leads['Specialization'].fillna('Unspecified')

In [28]:
def categories(x):
    
    if x in ['Finance Management','Human Resource Management','Marketing Management','Operations Management',
             'IT Projects Management', 'Supply Chain Management', 'Healthcare Management', 'Hospitality Management',
             'Retail Management']:
        return 'Management Specializations'
    elif x == 'Business Administration':
        return 'Business Administration'
    elif x == 'Banking, Investment And Insurance':
        return 'Banking'
    elif x == 'Unspecified':
        return 'Unspecified'
    else:
        return 'Others'

leads['Specialization'] = leads['Specialization'].apply(categories)
leads['Specialization'].value_counts(normalize=True)

Management Specializations    0.460281
Unspecified                   0.365801
Others                        0.093723
Business Administration       0.043615
Banking                       0.036580
Name: Specialization, dtype: float64

In [29]:
# Checking null value percentage in each column

round(100*(leads.isnull().sum())/len(leads),2)

Lead Origin                                       0.00
Lead Source                                       0.39
Do Not Email                                      0.00
Do Not Call                                       0.00
Converted                                         0.00
TotalVisits                                       1.48
Total Time Spent on Website                       0.00
Page Views Per Visit                              1.48
Last Activity                                     1.11
Specialization                                    0.00
What is your current occupation                  29.11
What matters most to you in choosing a course    29.32
Search                                            0.00
Newspaper Article                                 0.00
X Education Forums                                0.00
Newspaper                                         0.00
Digital Advertisement                             0.00
Through Recommendations                           0.00
Tags      

**City**

In [30]:
# Checking the unique categories in 'City'

leads['City'].value_counts(normalize=True,dropna=False)

NaN                            0.397078
Mumbai                         0.348701
Thane & Outskirts              0.081385
Other Cities                   0.074242
Other Cities of Maharashtra    0.049459
Other Metro Cities             0.041126
Tier II Cities                 0.008009
Name: City, dtype: float64

In [31]:
def city_categories(x):
    
    if x == 'Mumbai':
        return 'Mumbai'
    elif x in ['Thane & Outskirts', 'Other Cities of Maharashtra']:
        return 'Other Cities of Maharashtra'
    elif x == 'Other Metro Cities':
        return 'Metro Cities'
    else:
        return 'Other Cities'

leads['City'] = leads['City'].apply(city_categories)
leads['City'].value_counts(normalize=True)

Other Cities                   0.479329
Mumbai                         0.348701
Other Cities of Maharashtra    0.130844
Metro Cities                   0.041126
Name: City, dtype: float64

**Tags**

In [32]:
leads['Tags'].value_counts(normalize=True,dropna=False)

NaN                                                  0.362879
Will revert after reading the email                  0.224242
Ringing                                              0.130195
Interested in other courses                          0.055519
Already a student                                    0.050325
Closed by Horizzon                                   0.038745
switched off                                         0.025974
Busy                                                 0.020130
Lost to EINS                                         0.018939
Not doing further education                          0.015693
Interested  in full time MBA                         0.012662
Graduation in progress                               0.012013
invalid number                                       0.008983
Diploma holder (Not Eligible)                        0.006818
wrong number given                                   0.005087
opp hangup                                           0.003571
number n

In [33]:
# Checking null value percentage in each column

round(100*(leads.isnull().sum())/len(leads),2)

Lead Origin                                       0.00
Lead Source                                       0.39
Do Not Email                                      0.00
Do Not Call                                       0.00
Converted                                         0.00
TotalVisits                                       1.48
Total Time Spent on Website                       0.00
Page Views Per Visit                              1.48
Last Activity                                     1.11
Specialization                                    0.00
What is your current occupation                  29.11
What matters most to you in choosing a course    29.32
Search                                            0.00
Newspaper Article                                 0.00
X Education Forums                                0.00
Newspaper                                         0.00
Digital Advertisement                             0.00
Through Recommendations                           0.00
Tags      

In [34]:
leads.shape

(9240, 22)

**What is your current occupation**

In [35]:
# Checking the unique value counts for the column 'What is your current occupation'

leads['What is your current occupation'].value_counts(normalize=True, dropna=False)

Unemployed              0.606061
NaN                     0.291126
Working Professional    0.076407
Student                 0.022727
Other                   0.001732
Housewife               0.001082
Businessman             0.000866
Name: What is your current occupation, dtype: float64

In [36]:
# Replacing NaN with "Undefined" in the column 'What is your current occupation' 

leads['What is your current occupation'] = leads['What is your current occupation'].replace(np.nan,"Undefined")

leads['What is your current occupation'].isnull().sum()

0

In [37]:
leads['What is your current occupation'].value_counts(normalize=True, dropna=False)

Unemployed              0.606061
Undefined               0.291126
Working Professional    0.076407
Student                 0.022727
Other                   0.001732
Housewife               0.001082
Businessman             0.000866
Name: What is your current occupation, dtype: float64

**What matters most to you in choosing a course**

In [38]:
# Checking the unique value counts for the column 'What matters most to you in choosing a course'

leads['What matters most to you in choosing a course'].value_counts(dropna=False)

Better Career Prospects      6528
NaN                          2709
Flexibility & Convenience       2
Other                           1
Name: What matters most to you in choosing a course, dtype: int64

In [39]:
# Replacing NaN with "Other" in the column 'What matters most to you in choosing a course' 

leads['What matters most to you in choosing a course'] = leads['What matters most to you in choosing a course'].replace(np.nan,"Other")

leads['What matters most to you in choosing a course'].isnull().sum()

0

In [40]:
leads['What matters most to you in choosing a course'].value_counts(dropna=False)

Better Career Prospects      6528
Other                        2710
Flexibility & Convenience       2
Name: What matters most to you in choosing a course, dtype: int64

In [41]:
# Checking null value percentage in each column

round(100*(leads.isnull().sum())/len(leads),2)

Lead Origin                                       0.00
Lead Source                                       0.39
Do Not Email                                      0.00
Do Not Call                                       0.00
Converted                                         0.00
TotalVisits                                       1.48
Total Time Spent on Website                       0.00
Page Views Per Visit                              1.48
Last Activity                                     1.11
Specialization                                    0.00
What is your current occupation                   0.00
What matters most to you in choosing a course     0.00
Search                                            0.00
Newspaper Article                                 0.00
X Education Forums                                0.00
Newspaper                                         0.00
Digital Advertisement                             0.00
Through Recommendations                           0.00
Tags      

**There are columns which have very less missing values. Dropping the rows corresponding to the columns having very low missing values.**

In [ ]:
leads['Lead Source'].value_counts(dropna=False)

In [ ]:
# Dropping the null valued rows in the column 'Lead Source'

leads=leads[~pd.isnull(leads['Lead Source'])]



In [ ]:
# Combining categories


def Lead_Source_categories(x):
    
    if x in  ['Google','google']:
        return 'Google'
    elif x in ['Facebook','Social Media']:
        return 'Social Media'
    elif x == 'Direct Traffic':
        return 'Direct Traffic'
    elif x == 'Organic Search':
        return 'Organic Search'
    else:
        return 'Others'

leads['Lead Source'] = leads['Lead Source'].apply(Lead_Source_categories)
leads['Lead Source'].value_counts(normalize=True)

In [ ]:
# # Replacing null Values and combining categories

# leads['Specialization'] = leads['Specialization'].replace(np.nan,'Others')
# leads['Specialization'] = leads['Specialization'].replace('google','Google')
# leads['Specialization'] = leads['Specialization'].replace('Facebook','Social Media')
# leads['Specialization'] = leads['Specialization'].replace(['bing','Click2call','Press_Release',
#                                                      'youtubechannel','welearnblog_Home',
#                                                      'WeLearn','blog','Pay per Click Ads',
#                                                     'testone','NC_EDM'] ,'Others')  

In [ ]:
leads['TotalVisits'].isnull().sum()

In [ ]:
print(leads['Page Views Per Visit'].mean())
print(leads['Page Views Per Visit'].median())

In [ ]:
leads['TotalVisits'] = leads['TotalVisits'].fillna(leads['TotalVisits'].median())
leads['TotalVisits'].isnull().sum()

In [ ]:
leads['Page Views Per Visit'].isnull().sum()

In [ ]:
print(leads['Page Views Per Visit'].mean())
print(leads['Page Views Per Visit'].median())

In [ ]:
leads['Page Views Per Visit'] = leads['Page Views Per Visit'].fillna(leads['Page Views Per Visit'].median())
leads['Page Views Per Visit'].isnull().sum()

In [ ]:
leads['Last Activity'].value_counts(dropna=False, normalize=True)

In [ ]:
leads['Last Notable Activity'].value_counts(dropna=False, normalize=True)

In [ ]:
# 'Last Activity' and 'Last Notable Activity' yield almost same results and 'Last Notable Activity' has no missing value.
# Hence, dropping 'Last Activity' column

leads.drop(['Last Activity'], 1, inplace = True)
leads.shape

In [ ]:
# Combining categories in the column 'Last Notable Activity's

leads['Last Notable Activity'] = leads['Last Notable Activity'].replace(['Olark Chat Conversation','Email Link Clicked', 
                                                                         'Email Bounced', 'Unsubscribed', 'Unreachable', 
                                                                         'Had a Phone Conversation', 'Email Marked Spam',
                                                                         'Approached upfront', 'Resubscribed to emails', 
                                                                         'View in browser link Clicked', 'Form Submitted on Website',
                                                                         'Email Received'], 'Other Notable Activity')

leads['Last Notable Activity'].value_counts()
    

                                                                        

In [ ]:
print(leads['Newspaper'].value_counts(normalize=True))

print(leads['Newspaper Article'].value_counts(normalize=True))

**'Newspaper' and 'Newspaper Article' have almost similar results, and more than 99.9% values are marked 'No'. So dropping these two columns as it will give nothing significant**

In [ ]:
# Dropping column 'Newspaper'

leads.drop(['Newspaper','Newspaper Article'], 1, inplace = True)
leads.shape

In [ ]:
# Checking null value percentage

round(100*(leads.isnull().sum())/len(leads),2)

In [ ]:
leads.columns

In [ ]:
var_list = ['Lead Origin', 'Lead Source', 'Do Not Email', 'Do Not Call',
       'Converted', 'Specialization',
       'What is your current occupation',
       'What matters most to you in choosing a course', 'Search',
       'X Education Forums', 'Digital Advertisement',
       'Through Recommendations', 'Tags', 'City',
       'A free copy of Mastering The Interview', 'Last Notable Activity']


for i in var_list:
    ans=leads[i].value_counts(dropna=False)
    print(f"{i} Count:\n\n{ans}",end='\n\n\n')

# **Segment - II : Exploratory Data Analysis**

## Segregating continuous and categorical variables

In [ ]:
leads.columns

In [ ]:
leads.info()

In [ ]:
leads.head()

In [ ]:
# Separating continuous and categorical columns

cont_cols=['TotalVisits', 'Total Time Spent on Website', 'Page Views Per Visit']

cat_cols = ['Lead Origin', 'Lead Source', 'Do Not Email', 'Do Not Call', 'Specialization', 'What is your current occupation',
       'What matters most to you in choosing a course', 'Search', 'X Education Forums', 'Digital Advertisement',
       'Through Recommendations', 'Tags', 'City', 'A free copy of Mastering The Interview', 'Last Notable Activity']

target=['Converted']

len(cont_cols)+len(cat_cols)+len(target)

In [ ]:
# Finding the value count for each category of the categorical columns

for i in cat_cols:
    ans=leads[i].value_counts()
    print(f"{i} Count:\n\n{ans}",end='\n\n\n')
    

**Checking data imbalance in target variable**

In [ ]:
leads.Converted.value_counts(normalize=True, dropna=False)

## Univariate Analysis

### Univariate Analysis on Numerical columns

In [ ]:
# Plotting distribution curve for continuous columns

plt.figure(figsize=(18,5))
for num in enumerate(cont_cols):
    plt.subplot(len(cont_cols)//2,3,num[0]+1)
    sns.distplot(leads[num[1]])
plt.show()

In [ ]:
# Plotting boxplot on continuous columns for outlier detection

plt.figure(figsize=(18,5))
for num in enumerate(cont_cols):
    plt.subplot(len(cont_cols)//2,3,num[0]+1)
    sns.boxplot(leads[num[1]])
plt.show()

In [ ]:
len(cat_cols)

### Univariate Analysis on Categorical columns

In [ ]:
# Plotting Countplot for univariate analysis of categorical columns
 

plt.figure(figsize=(35,80))
for cat in enumerate(cat_cols):
    plt.subplot(8,2,cat[0]+1)
    sns.countplot(y=leads[cat[1]])
    plt.ylabel(cat[1], fontsize=18, fontweight=5, color='Brown')
    plt.xticks(rotation=90, fontsize=15)
    plt.yticks(rotation=0, fontsize=15)
plt.show()


## Bivariate Analysis

### Bivariate analysis of numerical variables with respect to target variable 'Converted'

In [ ]:
# Analysing all continuous columns with respect to the target variable 'Converted' using scatter plot

plt.figure(figsize=(18,5))
for num in enumerate(cont_cols):
    plt.subplot(len(cont_cols)//2,3,num[0]+1)
    plt.title(num[1] +" vs Converted", fontsize=15, fontweight=5, color='Brown')
    sns.scatterplot(leads[num[1]],leads["Converted"])    
plt.show()

### Bivariate analysis of categorical variables with respect to target variable 'cnt'


In [ ]:
# Analysing all categorical columns with respect to the target variable 'Converted' using bar plot

for cat in enumerate(cat_cols):
    plt.subplots(1,1,figsize=(10,5))
    plt.title(cat[1] +" vs Converted", fontsize=15, fontweight=5, color='Brown')
    sns.barplot(leads[cat[1]],leads["Converted"])
#     plt.xlabel(cat[1], fontsize=12, fontweight=5, color='Brown')
#     plt.ylabel("Converted", fontsize=12, fontweight=5, color='Brown')
    plt.xticks(rotation=90)
plt.show()


In [ ]:
# Analysing all categorical columns with respect to the target variable 'Converted' using bar plot

plt.figure(figsize=(30,100))
for cat in enumerate(cat_cols):
    plt.subplot(8,2,cat[0]+1)
    sns.barplot(leads[cat[1]],leads["Converted"])
    plt.xlabel(cat[1], fontsize=12, fontweight=5, color='Brown')
    plt.ylabel("Converted", fontsize=12, fontweight=5, color='Brown')
    plt.xticks(rotation=45)
plt.show()


### Correlation of  numerical variable

In [ ]:
# Plotting a Heatmap for finding correlation of numerical variables, including the target variable.

plt.figure(figsize=(8,6))
sns.heatmap(leads[['TotalVisits', 'Total Time Spent on Website', 'Page Views Per Visit', 'Converted']].corr(),cmap="YlGn", annot=True)
plt.show()


In [ ]:
# Plotting a pairplot for checking correlation of numerical variables including the target variable.

sns.pairplot(data=leads, vars=['TotalVisits', 'Total Time Spent on Website', 'Page Views Per Visit', 'Converted'])
plt.show()


## Multivariate Analysis

In [ ]:
leads.head()

# **Segment - III : Data Preparation For Model Building**

## Pre-processing steps

### Converting some binary variables (Yes/No) to 0/1

In [ ]:
# List of variables to map

varlist =  ['Do Not Email', 'Do Not Call', 'Search', 'X Education Forums', 'Newspaper','Digital Advertisement','Through Recommendations','A free copy of Mastering The Interview']

# Defining the map function
def binary_map(x):
    return x.map({'Yes': 1, "No": 0})

# Applying the function to the housing list
leads[varlist] = leads[varlist].apply(binary_map)